In [31]:
import json
import formulas
from openpyxl import load_workbook

In [2]:
wb = load_workbook(filename = 'current-central-steam-methane-reforming-without-co2-sequestration-version-aug22.xlsm')

/Users/rruizesparza/.local/share/virtualenvs/h2a-Q9QHEO75/lib/python3.11/site-packages/openpyxl/reader/drawings.py:29: UserWarning: DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.
  warn("DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.")
/Users/rruizesparza/.local/share/virtualenvs/h2a-Q9QHEO75/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/rruizesparza/.local/share/virtualenvs/h2a-Q9QHEO75/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
wb["Title"]["B2"].value

'Central Hydrogen Production - Project Information'

In [4]:
ws = wb.active

In [5]:
ws.title

'Input_Sheet_Template'

In [6]:
len(wb.defined_names.keys())

333

In [7]:
wb.defined_names["output_kgpday"].value

'Input_Sheet_Template!$C$27'

In [8]:
sheet, cell = wb.defined_names["output_kgpday"].value.split("!")

In [9]:
wb[sheet][cell].value

'=design_cap*cap_factor'

In [23]:
def fmt(val, sheet=""):
    return {
        "sheet": sheet,
        "data": val
    }
    
def lookup(name):
    raw_coord = wb.defined_names[name].value
    parts = raw_coord.split("!")
    if(len(parts) != 2):
        return fmt(raw_coord)
    sheet, cell = parts
    
    # Handle sheet names wrapped with quotes
    if sheet.startswith("'") and sheet.endswith("'"):
        sheet = sheet[1:-1]

    # Handle ranges
    if ":" in cell:
        start, end = cell.split(":")
        series = wb[sheet][start : end]
        values = [cell[0].value for cell in series]
        return fmt(values, sheet)
    
    # Single cell values
    return fmt(wb[sheet][cell].value, sheet)

In [11]:
lookup("design_cap")

   Input_Sheet_Template!$C$26


'=ProcessFlow!V12*sensitivity_02'

In [12]:
lookup("sensitivity_02")

   'Tornado Charts & Risk Analysis'!$D$48


'=+OFFSET(D50,$B$48,0)'

In [35]:
names_dict = {}
flat_name_dict = {}
for name in wb.defined_names:
    term = lookup(name)
    flat_name_dict[name] = term["data"]
    names_dict.setdefault(term["sheet"], []).append({ name: term["data"] })

with open('names.json', 'w', encoding='utf-8') as f:
    json.dump(names_dict, f, ensure_ascii=False, indent=4)

In [41]:
func = formulas.Parser().ast(flat_name_dict["acre_cost"])[1].compile()
resolved_inputs = []
for cell in list(func.inputs):
    resolved_inputs.append(wb["Input_Sheet_Template"][cell].value)
resolved_inputs

[50000, '=CPIinflator']